Using the shot chart avaialable for each nba player via the nba api to create a machine learning routine that should be capable of predicting whether or not a basket will go in.

Variables include position, coverage, and time. 

Making it one big function that way as soon as you end the player, its done

In [5]:
from nba_py import player
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
np.random.seed(4)  #replication purposes.
import matplotlib.pyplot as plt
from sklearn import preprocessing, cross_validation, neighbors
from sklearn.neural_network import MLPClassifier
from nba_py import shotchart
%matplotlib inline

In [2]:
import nba_py

In [3]:
day = nba_py.Scoreboard()

In [4]:
day.line_score()

,GAME_DATE_EST,GAME_SEQUENCE,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY_NAME,TEAM_WINS_LOSSES,PTS_QTR1,PTS_QTR2,PTS_QTR3,PTS_QTR4,PTS_OT1,PTS_OT2,PTS_OT3,PTS_OT4,PTS_OT5,PTS_OT6,PTS_OT7,PTS_OT8,PTS_OT9,PTS_OT10,PTS,FG_PCT,FT_PCT,FG3_PCT,AST,REB,TOV
0,2018-05-06T00:00:00,1,0041700234,1610612744,GSW,Golden State,3-1,37,24,33,24,0,0,0,0,0,0,0,0,0,0,118,0.484,0.833,0.333,28,45,11
1,2018-05-06T00:00:00,1,0041700234,1610612740,NOP,New Orleans,1-3,22,32,19,19,0,0,0,0,0,0,0,0,0,0,92,0.364,0.800,0.154,17,51,19
2,2018-05-06T00:00:00,2,0041700224,1610612745,HOU,Houston,2-1,30,16,0,0,0,0,0,0,0,0,0,0,0,0,46,0.432,0.900,0.357,7,25,5
3,2018-05-06T00:00:00,2,0041700224,1610612762,UTA,Utah,1-2,23,14,0,0,0,0,0,0,0,0,0,0,0,0,37,0.389,0.750,0.273,7,17,3


In [6]:
players = player.PlayerList()

In [7]:
playersdf = players.info()

In [10]:
playersdf

,PERSON_ID,DISPLAY_LAST_COMMA_FIRST,DISPLAY_FIRST_LAST,ROSTERSTATUS,FROM_YEAR,TO_YEAR,PLAYERCODE,TEAM_ID,TEAM_CITY,TEAM_NAME,TEAM_ABBREVIATION,TEAM_CODE,GAMES_PLAYED_FLAG
0,203518,"Abrines, Alex",Alex Abrines,1,2016,2017,alex_abrines,1610612760,Oklahoma City,Thunder,OKC,thunder,Y
1,203112,"Acy, Quincy",Quincy Acy,1,2012,2017,quincy_acy,1610612751,Brooklyn,Nets,BKN,nets,Y
2,203500,"Adams, Steven",Steven Adams,1,2013,2017,steven_adams,1610612760,Oklahoma City,Thunder,OKC,thunder,Y
3,1628389,"Adebayo, Bam",Bam Adebayo,1,2017,2017,bam_adebayo,1610612748,Miami,Heat,MIA,heat,Y
4,201167,"Afflalo, Arron",Arron Afflalo,1,2007,2017,arron_afflalo,1610612753,Orlando,Magic,ORL,magic,Y
5,201582,"Ajinca, Alexis",Alexis Ajinca,1,2008,2017,alexis_ajinca,1610612740,New Orleans,Pelicans,NOP,pelicans,Y
6,202332,"Aldrich, Cole",Cole Aldrich,1,2010,2017,cole_aldrich,1610612750,Minnesota,Timberwolves,MIN,timberwolves,Y
7,200746,"Aldridge, LaMarcus",LaMarcus Aldridge,1,2006,2017,lamarcus_aldridge,1610612759,San Antonio,Spurs,SAS,spurs,Y
8,1628386,"Allen, Jarrett",Jarrett Allen,1,2017,2017,jarrett_allen,1610612751,Brooklyn,Nets,BKN,nets,Y
9,1628443,"Allen, Kadeem",Kadeem Allen,1,2017,2017,kadeem_allen,1610612738,Boston,Celtics,BOS,celtics,Y


In [ ]:
PIDs = playersdf['PERSON_ID'].valuues

In [11]:
stevenadams = 203500

In [13]:
chart = shotchart.ShotChart(stevenadams)
chartdf = chart.shot_chart()

In [14]:
chartdf

,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,EVENT_TYPE,ACTION_TYPE,SHOT_TYPE,SHOT_ZONE_BASIC,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM
0,Shot Chart Detail,0021700015,17,203500,Steven Adams,1610612760,Oklahoma City Thunder,1,10,34,Made Shot,Cutting Layup Shot,2PT Field Goal,Restricted Area,Center(C),Less Than 8 ft.,1,6,9,1,1,20171019,OKC,NYK
1,Shot Chart Detail,0021700015,311,203500,Steven Adams,1610612760,Oklahoma City Thunder,2,2,26,Made Shot,Putback Dunk Shot,2PT Field Goal,Restricted Area,Center(C),Less Than 8 ft.,0,2,-6,1,1,20171019,OKC,NYK
2,Shot Chart Detail,0021700015,438,203500,Steven Adams,1610612760,Oklahoma City Thunder,3,5,50,Made Shot,Alley Oop Dunk Shot,2PT Field Goal,Restricted Area,Center(C),Less Than 8 ft.,0,-3,-5,1,1,20171019,OKC,NYK
3,Shot Chart Detail,0021700015,457,203500,Steven Adams,1610612760,Oklahoma City Thunder,3,4,37,Made Shot,Alley Oop Layup shot,2PT Field Goal,Restricted Area,Center(C),Less Than 8 ft.,0,8,1,1,1,20171019,OKC,NYK
4,Shot Chart Detail,0021700015,595,203500,Steven Adams,1610612760,Oklahoma City Thunder,4,7,17,Made Shot,Hook Shot,2PT Field Goal,In The Paint (Non-RA),Center(C),Less Than 8 ft.,4,-3,41,1,1,20171019,OKC,NYK
5,Shot Chart Detail,0021700036,47,203500,Steven Adams,1610612760,Oklahoma City Thunder,1,7,36,Made Shot,Hook Shot,2PT Field Goal,In The Paint (Non-RA),Center(C),Less Than 8 ft.,7,-22,68,1,1,20171021,UTA,OKC
6,Shot Chart Detail,0021700036,54,203500,Steven Adams,1610612760,Oklahoma City Thunder,1,6,42,Made Shot,Jump Shot,2PT Field Goal,In The Paint (Non-RA),Center(C),8-16 ft.,12,35,115,1,1,20171021,UTA,OKC
7,Shot Chart Detail,0021700036,90,203500,Steven Adams,1610612760,Oklahoma City Thunder,1,4,6,Missed Shot,Turnaround Hook Shot,2PT Field Goal,In The Paint (Non-RA),Center(C),Less Than 8 ft.,7,-14,77,1,0,20171021,UTA,OKC
8,Shot Chart Detail,0021700036,108,203500,Steven Adams,1610612760,Oklahoma City Thunder,1,2,47,Missed Shot,Jump Shot,2PT Field Goal,In The Paint (Non-RA),Center(C),8-16 ft.,12,41,119,1,0,20171021,UTA,OKC
9,Shot Chart Detail,0021700036,425,203500,Steven Adams,1610612760,Oklahoma City Thunder,3,4,58,Missed Shot,Tip Layup Shot,2PT Field Goal,Restricted Area,Center(C),Less Than 8 ft.,0,0,-6,1,0,20171021,UTA,OKC


In [ ]:
for PID in PIDs:
    print(PID)

In [ ]:
def get_PID(first_name,last_name):
    """Obtains the player ID number of the desired player.
    
    Parameters
    -----------
    
    first_name: str
        First name of player
    last_name: str
        Last name of player
    
    Returns
    -------
    
    ID : float
        The player's ID number. 
    """
    
    playa = player.get_player(first_name=first_name,last_name=last_name)
    
    return playa.values[0]
def get_shotchart(PID):
    """Returns a dataframe of the shotchart.
    """
    chart = shotchart.ShotChart(PID)
    chartdf = chart.shot_chart()
    print("head")
    print(chartdf.head())
  #  importantstuff = chartdf[['LOC_X','LOC_Y','SHOT_DISTANCE','ACTION_TYPE','EVENT_TYPE']].copy()   #['LOC_X','LOC_Y','EVENT_TYPE']
    importantstuff = chartdf[['LOC_X','LOC_Y','ACTION_TYPE','EVENT_TYPE']].copy()
    importantstuff = pd.get_dummies(importantstuff)
    return importantstuff

In [ ]:
def analyze_player(first,last,plot=True,cluster=True,Classifier=True):
    """Wrapper function which does everything in one, 
    no need to specify each table specific to that player right away!
    
    Parameters
    -----------
    first and last name of the player, comma separated, string format. 
    
    and a plotting option to see the shot chart, and anything else if I think of it. 
    and a cluster option to try and use ML to predict whether or not a shot goes in!
    
    Returns
    -------
    
    Shot chart, clustering success, anything else
    """
    
    
    PID = get_PID(first,last)
    print(first,last,'PID',PID)
    playertable =  get_shotchart(PID)
    #print(playertable)
    if plot: 
        makes = playertable.loc[playertable['EVENT_TYPE_Missed Shot'] == 0]
        misses = playertable.loc[playertable['EVENT_TYPE_Missed Shot'] == 1]
        xmade = makes['LOC_X'].values/10
        ymade = makes['LOC_Y'].values/10
        xmissed = misses['LOC_X'].values/10
        ymissed = misses['LOC_Y'].values/10
        x = np.linspace(-60,60,100)
        #make a 3pt line for reference
        for i in range(len(x)):
            r = 22
            y = np.sqrt(r**2 - x**2)
        plt.plot(xmade,ymade,'mo',label = 'made')
        plt.plot(xmissed,ymissed,'yx',label = 'missed')
        plt.plot(x,y,'k-',label = '~3pt line')
        plt.xlim((-25, 25))  
        plt.ylim((0, 35))  

        plt.title("Shot Chart of " + first + " " + last)
        plt.xlabel('dist. from hoop (ft)')
        plt.ylabel('dist. from hoop (ft)')

        plt.legend()
        y = playertable[['EVENT_TYPE_Made Shot','EVENT_TYPE_Missed Shot']].values
        del playertable['EVENT_TYPE_Made Shot'],playertable['EVENT_TYPE_Missed Shot']
        X = playertable.values
    if cluster:
        X_train,X_test,y_train,y_test = cross_validation.train_test_split(X,y,test_size=0.2)
        clf = neighbors.KNeighborsClassifier()
        clf.fit(X_train,y_train)
        accuracy = clf.score(X_test,y_test)
        print("Accuracy of clustering:",100*accuracy,"%.")
    if Classifier:
        #try a NN, whatever
        #normalize rgith ere 
        from sklearn.preprocessing import MinMaxScaler
        scaler = MinMaxScaler()
        MinMaxScaler(copy=True, feature_range=(0, 1))

        scaler.fit(X)
        X = scaler.transform(X)
        X_train,X_test,y_train,y_test = cross_validation.train_test_split(X,y,test_size=0.2)
        model = MLPClassifier()
        model.shuffle = True
        model.fit(X_train,y_train)
        accuracy = model.score(X_test,y_test)

        print("Accuracy of MLP Classifier:",100*accuracy,"%.")


In [ ]:
analyze_player('Spencer','Dinwiddie')

In [ ]:
def analyze_players(plot=False,cluster=True,Classifier=True):
    """Wrapper function which does everything in one, 
    no need to specify each table specific to that player right away!
    

    Parameters
    -----------
    PIDSs. Array of all the active nba players, just to see which is best for fitting etc.     
    
    Returns
    -------
    
    Shot chart, clustering success, anything else
    """
    
    players = player.PlayerList() 
    
    playersdf = players.info()
    PIDs = playersdf['PERSON_ID'].values
    names = playersdf['DISPLAY_FIRST_LAST'].values
    for i in range(len(PIDs)):
        PID = PIDs[i]
        name = names[i]
        print(name)
        playertable =  get_shotchart(PID)
    #print(playertable)
        if plot: 
            makes = playertable.loc[playertable['EVENT_TYPE_Missed Shot'] == 0]
            misses = playertable.loc[playertable['EVENT_TYPE_Missed Shot'] == 1]
            xmade = makes['LOC_X'].values/10
            ymade = makes['LOC_Y'].values/10
            xmissed = misses['LOC_X'].values/10
            ymissed = misses['LOC_Y'].values/10
            x = np.linspace(-60,60,100)
        #make a 3pt line for reference
            for i in range(len(x)):
                r = 22
                y = np.sqrt(r**2 - x**2)
            plt.plot(xmade,ymade,'mo',label = 'made')
            plt.plot(xmissed,ymissed,'yx',label = 'missed')
            plt.plot(x,y,'k-',label = '~3pt line')
            plt.xlim((-25, 25))  
            plt.ylim((0, 35))  

            plt.title("Shot Chart of " + first + " " + last)
            plt.xlabel('dist. from hoop (ft)')
            plt.ylabel('dist. from hoop (ft)')

            plt.legend()
        y = playertable[['EVENT_TYPE_Made Shot','EVENT_TYPE_Missed Shot']].values
        del playertable['EVENT_TYPE_Made Shot'],playertable['EVENT_TYPE_Missed Shot']
        X = playertable.values
        if cluster:
            X_train,X_test,y_train,y_test = cross_validation.train_test_split(X,y,test_size=0.2)
            clf = neighbors.KNeighborsClassifier()
            clf.fit(X_train,y_train)
            accuracy = clf.score(X_test,y_test)
            print("Accuracy of clustering:",100*accuracy,"%.")
        if Classifier:
        #try a NN, whatever
        #normalize rgith ere 
            from sklearn.preprocessing import MinMaxScaler
            scaler = MinMaxScaler()
            MinMaxScaler(copy=True, feature_range=(0, 1))

            scaler.fit(X)
            X = scaler.transform(X)
            X_train,X_test,y_train,y_test = cross_validation.train_test_split(X,y,test_size=0.2)
            model = MLPClassifier()
            model.shuffle = True
            model.fit(X_train,y_train)
            accuracy = model.score(X_test,y_test)

            print("Accuracy of MLP Classifier:",100*accuracy,"%.")


In [ ]:
analyze_players()

In [ ]:
analyze_player('Alexis','Ajinca')

If I had to guess these lines are on the line, so approximately speaking 220 of these units corresponds to 22 feet. Ie divide by 10 to obtain feet. Doing this now!

In [ ]:
plt.plot(x,y)